In [ ]:
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
mcc = MouseConnectivityCache()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import pathlib
import os
from allensdk.api.queries.image_download_api import ImageDownloadApi
from allensdk.api.queries.svg_api import SvgApi
from allensdk.config.manifest import Manifest

In [ ]:
#load ROIs from LCM datasets from registration data
reg_dir = pathlib.Path('/nemo/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/allenccf/allen_ccf_coord')
ROI_path = pathlib.Path('/nemo/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/rois')
ROI_table = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/allenccf/ROI_vol.pkl')
allen_converted_path = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/allenccf/allen_ccf_coord')
download_allen = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/Allen_Connectivity')
reads_path = pathlib.Path("/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq/Final_processed_sequences")
tube_to_RT = pathlib.Path("/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/tube_to_RT.csv")

In [ ]:
#allen anterograde tracing datasets with more than 75% injection site AUDp
experiment_id_a = 120491896 # AUDp
experiment_id_b =116903230 # AUDp, AUDpo, AUDd, AUDv
experiment_id_c =100149109 # AUDp and AUDd
#injection volumes to normalise to (mm3)
expt_a_inj_vol = 0.097
expt_b_inj_vol = 0.114
expt_c_inj_vol= 0.073
# get projection density for each anterograde tracing expt: values are sum of projecting pixels per voxel.
expt_a, pd_a_info = mcc.get_projection_density(experiment_id_a)
expt_b, pd_b_info = mcc.get_projection_density(experiment_id_b)
expt_c, pd_c_info = mcc.get_projection_density(experiment_id_c)

In [ ]:
#re-get registration info on each ROI
#iterate through individual slices, take average difference in in coordinates in z (which is x axes in allen ccf) for last slice (slice s001 for brain 1), take average of previous slices
add_z = pd.DataFrame(columns=['slice', 'amountz'], dtype=int)
#need to change for mega thick last bit of cortex section, so extend ROI through 3slices
for file in os.listdir(allen_converted_path):
    if file.startswith('allen_ccf_converted_'):
        slice_name = file[20:24]
        slicenum = int(file[21:24])
        if slice_name != 's001' and slice_name != 's012' and slice_name != 's048' and slice_name != 's053':
            slice_before= slicenum-1
            if slice_before >9:
                slicebefore_name = f's0{slice_before}' 
            if slice_before<10:
                slicebefore_name = f's00{slice_before}' 
            [x1a, y1a, z1a, one1] = np.load(allen_converted_path/file)
            [x2a, y2a, z2a, one2] = np.load(allen_converted_path/f'allen_ccf_converted_{slicebefore_name}.npy')
            dif = np.average(x2a.flatten()-x1a.flatten())
            add_z= add_z.append({'slice': slice_name, 'amountz': dif},ignore_index=True)
#for slices where the one's before are missing, extend them by the mean of slice z extensions for the others
s001_add =add_z['amountz'].mean()
add_z= add_z.append({'slice': 's001', 'amountz': s001_add},ignore_index=True)
s012_add =add_z['amountz'].mean()
add_z= add_z.append({'slice': 's012', 'amountz': s012_add},ignore_index=True)
s048_add =add_z['amountz'].mean()
add_z= add_z.append({'slice': 's048', 'amountz': s048_add},ignore_index=True)
s053_add =add_z['amountz'].mean()
add_z= add_z.append({'slice': 's053', 'amountz': s053_add},ignore_index=True)

In [ ]:
#roipath = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/rois')
ROI_vol_expt_a=pd.DataFrame()

for region in os.listdir(ROI_path):
    if region.startswith("s0"):
        slicename = region[0:4]
        tube = region[5:len(region)].split('TUBE', 1)[1]
        tube =tube[:-4]
        [xa, ya, za, one] = np.load(allen_converted_path/f'allen_ccf_converted_{slicename}.npy')
        roi = plt.imread(ROI_path/f'{region}')
        allencoord_roiya = roi*ya
        allencoord_roiza = roi*za
#use shoelace formula to define area of polygon given xy coordinates then calculate volume of each LCM roi
        calcz = allencoord_roiza[allencoord_roiza!= 0]
        calcy = allencoord_roiya[allencoord_roiya!= 0]
        area_roi = 0.5*np.abs(np.dot(calcz,np.roll(calcy,1))-np.dot(calcy,np.roll(calcz,1)))
        z_to_add = add_z.loc[add_z['slice'] == slicename, 'amountz'].iloc[0]
        vol_roi = area_roi*-z_to_add
#convert the x, y, z coordinates to pixel
        pixcoord = []
        for i, axis in enumerate([xa, ya, za]):
            pixel = np.array(np.round(axis/25), dtype=int)
            pixel[pixel <0] = 0
            pixel[pixel >= expt_a.shape[i]] = 0
            pixcoord.append(pixel)

# use projection expt to convert each pixel to projection density

        registered_slice = np.zeros(xa.shape, dtype = expt_a.dtype)
        expt_a_roi=expt_a[pixcoord[0].flatten(),
                      pixcoord[1].flatten(),
                      pixcoord[2].flatten()].reshape(registered_slice.shape)
        ROIanno = expt_a_roi*roi
#iterate image by z slices, each additional z, annotate then add to list
        slices = -round(z_to_add/25)
        for x in range(slices):
            if x >0:
                newz = pixcoord[0]-x #changed from plus to minus as going backwards
                slice = expt_a[newz.flatten(), pixcoord[1].flatten(), pixcoord[2].flatten()].reshape(registered_slice.shape)
                ROIanno_add = slice*roi
                ROIanno = np.append(ROIanno, ROIanno_add)
                sum_projpixels_per_voxel = ROIanno.sum()
        ROI_vol_expt_a= ROI_vol_expt_a.append({'slice': slicename, 'tube': tube, 'z_added': z_to_add, 'vol (um3)': vol_roi, 'projection_density_pix': ROIanno, 'sum_projpixels_per_voxel': sum_projpixels_per_voxel},ignore_index=True)




In [ ]:
#combine volumes for LCM
finalpix_expt_a = pd.DataFrame(columns=['tube', 'projection_density_summed', 'vol (um3)'], dtype=int)
result = ROI_vol_expt_a.groupby(['tube']).agg(', '.join).reset_index()
for row, tube in result['tube'].iteritems():
    newdf = ROI_vol_expt_a[ROI_vol_expt_a['tube']==tube].reset_index()
    #for count, value in enumerate(newdf):
    for r, t in newdf['tube'].iteritems():
        if r ==0 :
            projection_density_summed = newdf.loc[r, 'sum_projpixels_per_voxel']
            vol = newdf.loc[r, 'vol (um3)']
        if r > 0:
            projection_density_summed = projection_density_summed + newdf.loc[r, 'sum_projpixels_per_voxel']
            vol = vol + newdf.loc[r, 'vol (um3)']
    finalpix_expt_a= finalpix_expt_a.append({'tube': tube, 'projection_density_summed': projection_density_summed, 'vol (um3)': vol},ignore_index=True)
finalpix_expt_a.to_pickle('mouse_connectivity/finalpix_expt_a.pkl')
    

In [ ]:

#roipath = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/rois')
ROI_vol_expt_b=pd.DataFrame()

for region in os.listdir(ROI_path):
    if region.startswith("s0"):
        slicename = region[0:4]
        tube = region[5:len(region)].split('TUBE', 1)[1]
        tube =tube[:-4]
        [xa, ya, za, one] = np.load(allen_converted_path/f'allen_ccf_converted_{slicename}.npy')
        roi = plt.imread(ROI_path/f'{region}')
        allencoord_roiya = roi*ya
        allencoord_roiza = roi*za
#use shoelace formula to define area of polygon given xy coordinates then calculate volume of each LCM roi
        calcz = allencoord_roiza[allencoord_roiza!= 0]
        calcy = allencoord_roiya[allencoord_roiya!= 0]
        area_roi = 0.5*np.abs(np.dot(calcz,np.roll(calcy,1))-np.dot(calcy,np.roll(calcz,1)))
        z_to_add = add_z.loc[add_z['slice'] == slicename, 'amountz'].iloc[0]
        vol_roi = area_roi*-z_to_add
#convert the x, y, z coordinates to pixel
        pixcoord = []
        for i, axis in enumerate([xa, ya, za]):
            pixel = np.array(np.round(axis/25), dtype=int)
            pixel[pixel <0] = 0
            pixel[pixel >= expt_a.shape[i]] = 0
            pixcoord.append(pixel)

# use projection expt to convert each pixel to projection density

        registered_slice = np.zeros(xa.shape, dtype = expt_a.dtype)
        expt_b_roi=expt_b[pixcoord[0].flatten(),
                      pixcoord[1].flatten(),
                      pixcoord[2].flatten()].reshape(registered_slice.shape)
        ROIanno = expt_b_roi*roi
#iterate image by z slices, each additional z, annotate then add to list
        slices = -round(z_to_add/25)
        for x in range(slices):
            if x >0:
                newz = pixcoord[0]-x #changed from plus to minus as going backwards
                slice = expt_b[newz.flatten(), pixcoord[1].flatten(), pixcoord[2].flatten()].reshape(registered_slice.shape)
                ROIanno_add = slice*roi
                ROIanno = np.append(ROIanno, ROIanno_add)
                sum_projpixels_per_voxel = ROIanno.sum()
        ROI_vol_expt_b= ROI_vol_expt_b.append({'slice': slicename, 'tube': tube, 'z_added': z_to_add, 'vol (um3)': vol_roi, 'projection_density_pix': ROIanno, 'sum_projpixels_per_voxel': sum_projpixels_per_voxel},ignore_index=True)

#combine volumes for LCM
finalpix_expt_b = pd.DataFrame(columns=['tube', 'projection_density_summed', 'vol (um3)'], dtype=int)
result = ROI_vol_expt_b.groupby(['tube']).agg(', '.join).reset_index()
for row, tube in result['tube'].iteritems():
    newdf = ROI_vol_expt_b[ROI_vol_expt_b['tube']==tube].reset_index()
    #for count, value in enumerate(newdf):
    for r, t in newdf['tube'].iteritems():
        if r ==0 :
            projection_density_summed = newdf.loc[r, 'sum_projpixels_per_voxel']
            vol = newdf.loc[r, 'vol (um3)']
        if r > 0:
            projection_density_summed = projection_density_summed + newdf.loc[r, 'sum_projpixels_per_voxel']
            vol = vol + newdf.loc[r, 'vol (um3)']
    finalpix_expt_b= finalpix_expt_b.append({'tube': tube, 'projection_density_summed': projection_density_summed, 'vol (um3)': vol},ignore_index=True)
finalpix_expt_b.to_pickle('mouse_connectivity/finalpix_expt_b.pkl')


In [ ]:

#roipath = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/rois')
ROI_vol_expt_c=pd.DataFrame()

for region in os.listdir(ROI_path):
    if region.startswith("s0"):
        slicename = region[0:4]
        tube = region[5:len(region)].split('TUBE', 1)[1]
        tube =tube[:-4]
        [xa, ya, za, one] = np.load(allen_converted_path/f'allen_ccf_converted_{slicename}.npy')
        roi = plt.imread(ROI_path/f'{region}')
        allencoord_roiya = roi*ya
        allencoord_roiza = roi*za
#use shoelace formula to define area of polygon given xy coordinates then calculate volume of each LCM roi
        calcz = allencoord_roiza[allencoord_roiza!= 0]
        calcy = allencoord_roiya[allencoord_roiya!= 0]
        area_roi = 0.5*np.abs(np.dot(calcz,np.roll(calcy,1))-np.dot(calcy,np.roll(calcz,1)))
        z_to_add = add_z.loc[add_z['slice'] == slicename, 'amountz'].iloc[0]
        vol_roi = area_roi*-z_to_add
#convert the x, y, z coordinates to pixel
        pixcoord = []
        for i, axis in enumerate([xa, ya, za]):
            pixel = np.array(np.round(axis/25), dtype=int)
            pixel[pixel <0] = 0
            pixel[pixel >= expt_a.shape[i]] = 0
            pixcoord.append(pixel)

# use projection expt to convert each pixel to projection density

        registered_slice = np.zeros(xa.shape, dtype = expt_a.dtype)
        expt_c_roi=expt_c[pixcoord[0].flatten(),
                      pixcoord[1].flatten(),
                      pixcoord[2].flatten()].reshape(registered_slice.shape)
        ROIanno = expt_c_roi*roi
#iterate image by z slices, each additional z, annotate then add to list
        slices = -round(z_to_add/25)
        for x in range(slices):
            if x >0:
                newz = pixcoord[0]-x #changed from plus to minus as going backwards
                slice = expt_c[newz.flatten(), pixcoord[1].flatten(), pixcoord[2].flatten()].reshape(registered_slice.shape)
                ROIanno_add = slice*roi
                ROIanno = np.append(ROIanno, ROIanno_add)
                sum_projpixels_per_voxel = ROIanno.sum()
        ROI_vol_expt_c= ROI_vol_expt_c.append({'slice': slicename, 'tube': tube, 'z_added': z_to_add, 'vol (um3)': vol_roi, 'projection_density_pix': ROIanno, 'sum_projpixels_per_voxel': sum_projpixels_per_voxel},ignore_index=True)

#combine volumes for LCM
finalpix_expt_c = pd.DataFrame(columns=['tube', 'projection_density_summed', 'vol (um3)'], dtype=int)
result = ROI_vol_expt_c.groupby(['tube']).agg(', '.join).reset_index()
for row, tube in result['tube'].iteritems():
    newdf = ROI_vol_expt_c[ROI_vol_expt_c['tube']==tube].reset_index()
    #for count, value in enumerate(newdf):
    for r, t in newdf['tube'].iteritems():
        if r ==0 :
            projection_density_summed = newdf.loc[r, 'sum_projpixels_per_voxel']
            vol = newdf.loc[r, 'vol (um3)']
        if r > 0:
            projection_density_summed = projection_density_summed + newdf.loc[r, 'sum_projpixels_per_voxel']
            vol = vol + newdf.loc[r, 'vol (um3)']
    finalpix_expt_c= finalpix_expt_c.append({'tube': tube, 'projection_density_summed': projection_density_summed, 'vol (um3)': vol},ignore_index=True)

    
finalpix_expt_c.to_pickle('mouse_connectivity/finalpix_expt_c.pkl')


In [ ]:
#plot relative reads to maximum
#load tables
finalpix_expt_a = pd.read_pickle('mouse_connectivity/finalpix_expt_a.pkl')
finalpix_expt_b = pd.read_pickle('mouse_connectivity/finalpix_expt_b.pkl')
finalpix_expt_c = pd.read_pickle('mouse_connectivity/finalpix_expt_c.pkl')

In [ ]:
#generate a table with total number of projection pixels normalised by injection volume for each expt.
all_projection_allen = pd.DataFrame().assign(Tube=finalpix_expt_a['tube'], ExptA = finalpix_expt_a['projection_density_summed']/expt_a_inj_vol, ExptB=finalpix_expt_b['projection_density_summed']/expt_b_inj_vol, ExptC=finalpix_expt_c['projection_density_summed']/expt_c_inj_vol)
all_projection_allen = all_projection_allen.set_index('Tube')                                         
                                         
 

In [ ]:
projection_comp = pd.DataFrame()

In [ ]:
projection_comp = pd.DataFrame(all_projection_allen.mean(axis=1), columns=['Allen'])
projection_comp['std_allen'] = all_projection_allen.std(axis=1)
RT_convert = pd.read_csv(tube_to_RT)
RT_convert = RT_convert.set_index('RT primer')

In [ ]:
#load neuron read counts for each tube
projection_comp['MAPseq_counts'] = 0
for file in os.listdir(reads_path):
    barcode_file = reads_path/file
    if barcode_file.stem.startswith('neuron_counts'):
        to_read = pd.read_csv(barcode_file) 
        df = to_read[to_read['counts'] > 2]
        counts = df['counts'].sum()
        #counts =to_read['counts'].sum()
        sample = barcode_file.stem.split("neuron_counts_BC", 1)[1]
        tube = RT_convert.loc[int(sample), 'Tube']
        projection_comp.loc[f'{tube}', 'MAPseq_counts'] = counts
projection_comp.index = projection_comp.index.astype(int)
projection_comp = projection_comp.sort_index()

In [ ]:
#normalise reads to leftmost MAPseq sample
div_amount = projection_comp.loc[65, 'MAPseq_counts']/projection_comp.loc[65, 'Allen']
projection_comp['Allen_norm'] = projection_comp['Allen']*div_amount
projection_comp['std_allen_norm'] = projection_comp['std_allen']*div_amount
projection_comp['MAPseq_norm'] = projection_comp['MAPseq_counts']/div_amount


In [ ]:
projection_comp['std_allen_norm'] = projection_comp['std_allen']*div_amount

In [ ]:
Neg_controls = [projection_comp.loc['1', 'Allen'], projection_comp.loc['2', 'Allen'], projection_comp.loc['3', 'Allen'], projection_comp.loc['4', 'Allen']]
neg_subtract =np.mean(Neg_controls)

In [ ]:
projection_comp['Allen_neg_subtract'] = projection_comp["Allen"]-neg_subtract

In [ ]:
x_list = []
for index in projection_comp.index:
    x_list.append(index)


In [ ]:
from scipy.stats import pearsonr
plt.scatter(np.log(projection_comp.Allen), np.log(projection_comp.MAPseq_counts))
plt.xlabel('log Allen pixels')
plt.ylabel('log MAPseq reads')
corr, _ = pearsonr(projection_comp.Allen, projection_comp.MAPseq_counts)
print(f'Pearsons correlation: {corr}')

In [ ]:
#all anterograde tracings together
fig = plt.figure(figsize=(20, 5))
ax1 = fig.add_subplot(111)
ax1.plot(x_list, all_projection_allen['ExptA']*div_amount, label='tracing expt a')
ax1.plot(x_list, all_projection_allen['ExptB']*div_amount, label='tracing expt b')
ax1.plot(x_list, all_projection_allen['ExptC']*div_amount, label='tracing expt c')
ax1.plot(x_list, projection_comp.MAPseq_counts, label='MAPseq')
#ax2 = plt.plot(x_list, projection_comp.Allen_norm)
plt.legend(loc='upper left')
plt.xticks(rotation = 90)
plt.xticks(np.arange(1, 94, 1.0))
plt.xlabel('Tubes')
plt.ylabel('Reads')
plt.yscale('log')


In [ ]:
plt.figure(figsize=(20, 5))
ax = fig.add_subplot(111)
b1 =plt.bar(x_list, projection_comp.MAPseq_counts, label="MAPseq_reads")
#b2 = plt.scatter(x_list, projection_comp.Allen, label="Allen_pixels")
berror = plt.errorbar(x_list, projection_comp.Allen_norm, yerr=projection_comp.std_allen_norm, fmt="o", label="Allen_pixels (normalised)", color='grey')
plt.yscale('log')
plt.xlabel('Tube')
plt.ylabel('Reads')
plt.legend()
plt.xticks(rotation = 90)
plt.xticks(np.arange(1, 94, 1.0))
plt.xlim(0, 94)
plt.show()

In [ ]:
#looking at cortex only from SSp (where expect signal)

cortex_from_SSP = [26, 27, 28,31, 32, 33, 34, 35, 36, 37, 45, 46, 47, 48, 53, 59, 60, 61, 62, 63, 64, 65, 73, 74, 75, 76, 77]
cortex = projection_comp.iloc[cortex_from_SSP]
x_list_cortex = []
for index in cortex.index:
    x_list_cortex.append(index)

plt.figure(figsize=(20, 5))
ax = fig.add_subplot(111)
b1 =plt.bar(x_list_cortex, cortex.MAPseq_counts, label="MAPseq_reads")
#b2 = plt.scatter(x_list, projection_comp.Allen, label="Allen_pixels")
berror = plt.errorbar(x_list_cortex, cortex.Allen_norm, yerr=cortex.std_allen_norm, fmt="o", label="Allen_pixels (normalised)", color='grey')
plt.yscale('log')
plt.xlabel('Tube')
plt.ylabel('Reads')
plt.legend()
plt.xticks(rotation = 90)
plt.xticks(np.arange(1, 94, 1.0))
plt.xlim(0, 94)
plt.title('cortex only')
plt.show()

In [ ]:
counts = pd.DataFrame()
for file in os.listdir(reads_path):
    barcode_file = reads_path/file
    if barcode_file.stem.startswith('neuron_counts'):
        sample = barcode_file.stem.split("neuron_counts_BC", 1)[1]
        if int(sample) in cortex_from_SSP:
            to_read = pd.read_csv(barcode_file)  
        counts =pd.concat([counts,to_read])
        #sample = barcode_file.stem.split("neuron_counts_BC", 1)[1]
        #tube = RT_convert.loc[int(sample), 'Tube']
        #projection_comp.loc[f'{tube}', 'MAPseq_counts'] = counts

In [ ]:
plt.hist(counts["counts"], bins=500)
plt.title('Number of UMI counts in cortex per barcode per sample post SSp', fontsize=12)
plt.yscale('log')

In [ ]:
ax = projection_comp.plot(y=["Allen", "MAPseq_norm"], use_index=True, figsize=(12, 6))
ax.set_ylabel('Summed Projection Density', fontsize=12)
ax.set_xlabel('Tube', fontsize=12)
ax.set_xticks(np.arange(0, len(all_projection_allen), 1))
ax.set_yscale('log')

plt.show()                                   


In [ ]:
#check injection mask
inj_a, inj_a_info = mcc.get_injection_density(experiment_id_b)
ROI_vol_expt_a=pd.DataFrame()

for region in os.listdir(ROI_path):
    if region.startswith("s0"):
        slicename = region[0:4]
        tube = region[5:len(region)].split('TUBE', 1)[1]
        tube =tube[:-4]
        [xa, ya, za, one] = np.load(allen_converted_path/f'allen_ccf_converted_{slicename}.npy')
        roi = plt.imread(ROI_path/f'{region}')
        allencoord_roiya = roi*ya
        allencoord_roiza = roi*za
#use shoelace formula to define area of polygon given xy coordinates then calculate volume of each LCM roi
        calcz = allencoord_roiza[allencoord_roiza!= 0]
        calcy = allencoord_roiya[allencoord_roiya!= 0]
        area_roi = 0.5*np.abs(np.dot(calcz,np.roll(calcy,1))-np.dot(calcy,np.roll(calcz,1)))
        z_to_add = add_z.loc[add_z['slice'] == slicename, 'amountz'].iloc[0]
        vol_roi = area_roi*-z_to_add
#convert the x, y, z coordinates to pixel
        pixcoord = []
        for i, axis in enumerate([xa, ya, za]):
            pixel = np.array(np.round(axis/25), dtype=int)
            pixel[pixel <0] = 0
            pixel[pixel >= inj_a.shape[i]] = 0
            pixcoord.append(pixel)

# use projection expt to convert each pixel to projection density

        registered_slice = np.zeros(xa.shape, dtype = inj_a.dtype)
        expt_a_roi=inj_a[pixcoord[0].flatten(),
                      pixcoord[1].flatten(),
                      pixcoord[2].flatten()].reshape(registered_slice.shape)
        ROIanno = expt_a_roi*roi
#iterate image by z slices, each additional z, annotate then add to list
        slices = -round(z_to_add/25)
        for x in range(slices):
            if x >0:
                newz = pixcoord[0]-x #changed from plus to minus as going backwards
                slice = inj_a[newz.flatten(), pixcoord[1].flatten(), pixcoord[2].flatten()].reshape(registered_slice.shape)
                ROIanno_add = slice*roi
                ROIanno = np.append(ROIanno, ROIanno_add)
                sum_projpixels_per_voxel = ROIanno.sum()
        ROI_vol_expt_a= ROI_vol_expt_a.append({'slice': slicename, 'tube': tube, 'z_added': z_to_add, 'vol (um3)': vol_roi, 'projection_density_pix': ROIanno, 'sum_projpixels_per_voxel': sum_projpixels_per_voxel},ignore_index=True)




In [ ]:
finalpix_expt_a = pd.DataFrame(columns=['tube', 'projection_density_summed', 'vol (um3)'], dtype=int)
result = ROI_vol_expt_a.groupby(['tube']).agg(', '.join).reset_index()
for row, tube in result['tube'].iteritems():
    newdf = ROI_vol_expt_a[ROI_vol_expt_a['tube']==tube].reset_index()
    #for count, value in enumerate(newdf):
    for r, t in newdf['tube'].iteritems():
        if r ==0 :
            projection_density_summed = newdf.loc[r, 'sum_projpixels_per_voxel']
            vol = newdf.loc[r, 'vol (um3)']
        if r > 0:
            projection_density_summed = projection_density_summed + newdf.loc[r, 'sum_projpixels_per_voxel']
            vol = vol + newdf.loc[r, 'vol (um3)']
    finalpix_expt_a= finalpix_expt_a.append({'tube': tube, 'projection_density_summed': projection_density_summed, 'vol (um3)': vol},ignore_index=True)
    
    
    finalpix_expt_a.to_pickle('mouse_connectivity/finalpix_expt_b_inj_pixels.pkl')
    

In [ ]:
#load neuron read counts for each tube
projection_comp['MAPseq_counts'] = 0
for file in os.listdir(reads_path):
    barcode_file = reads_path/file
    if barcode_file.stem.startswith('neuron_counts'):
        to_read = pd.read_csv(barcode_file)  
        counts =to_read['counts'].sum()
        sample = barcode_file.stem.split("neuron_counts_BC", 1)[1]
        tube = RT_convert.loc[int(sample), 'Tube']
        projection_comp.loc[f'{tube}', 'MAPseq_counts'] = counts

In [ ]:
finalpix_expt_a.to_pickle('mouse_connectivity/finalpix_expt_a_inj_pixels.pkl')

In [ ]:
finalpix_expt_a_proj = pd.read_pickle('mouse_connectivity/finalpix_expt_a.pkl')

In [ ]:
for i, row in finalpix_expt_a_proj.iterrows():
    print(row['tube'], row['projection_density_summed'])

In [ ]:
for i, row in finalpix_expt_a.iterrows():
    print(row['tube'], row['projection_density_summed'])

In [ ]:
barcode_file.stem.split("neuron_counts_BC", 1)[0]

In [ ]:
np.unique(inj_a)

In [ ]:
mean_allen

In [ ]:
#generate a table with total number of projection pixels normalised by injection volume for each expt.
all_projection_allen = pd.DataFrame().assign(Tube=finalpix_expt_d['tube'], ExptA = finalpix_expt_a['projection_density_summed'], ExptB=finalpix_expt_b['projection_density_summed'], ExptC=finalpix_expt_c['projection_density_summed'], ExptD=finalpix_expt_d['projection_density_summed'])

all_projection_allen = all_projection_allen.set_index('Tube')                                         
                                         
all_projection_allen                                             

In [ ]:
ax = all_projection_allen.plot(figsize=(12, 6))
ax.set_ylabel('Summed Projection Density', fontsize=12)
ax.set_xlabel('Tube', fontsize=12)
ax.set_xticks(np.arange(0, len(all_projection_allen), 1))

plt.show()                                   

In [ ]:
#load umi read count from neuron barcodes in MAPseq dataset
all_expt = pd.DataFrame(columns=['Tube', 'ExptA', 'ExptB', 'ExptC', 'ExptD'])

In [ ]:
for i, row in finalpix_expt_c.iterrows():
    print(row['projection_density_summed'], row['tube'], row['vol (um3)'])

In [ ]:
for i, row in finalpix.iterrows():
    print(row['projection_density_summed'], row['tube'], row['vol (um3)'])

In [ ]:
#load ROI info
ROI_vol = pd.read_pickle(ROI_table)

In [ ]:
1274289642.631726/11457280.182171874

In [ ]:
print(f'allenccf/allen_ccf_coord/allen_ccf_converted_{slicename}.npy')

In [ ]:
experiment_id = 100149109
#experiment_id =116903230
# projection density: number of projecting pixels / voxel volume
df, pd_info = mcc.get_projection_density(experiment_id)

# injection density: number of projecting pixels in injection site / voxel volume
ind, ind_info = mcc.get_injection_density(experiment_id)

# injection fraction: number of pixels in injection site / voxel volume
inf, inf_info = mcc.get_injection_fraction(experiment_id)

# data mask:
# binary mask indicating which voxels contain valid data
dm, dm_info = mcc.get_data_mask(experiment_id)

template, template_info = mcc.get_template_volume()
annot, annot_info = mcc.get_annotation_volume()

# in addition to the annotation volume, you can get binary masks for individual structures
# in this case, we'll get one for the isocortex
cortex_mask, cm_info = mcc.get_structure_mask(315)

print(pd_info)
print(df.shape, template.shape, annot.shape)

In [ ]:
# compute the maximum intensity projection (along the anterior-posterior axis) of the projection data
pd_mip = df.max(axis=0)
ind_mip = ind.max(axis=0)
inf_mip = inf.max(axis=0)

# show that slice of all volumes side-by-side
f, pr_axes = plt.subplots(1, 3, figsize=(15, 6))

pr_axes[0].imshow(pd_mip, cmap='hot', aspect='equal')
pr_axes[0].set_title("projection density MaxIP")

pr_axes[1].imshow(ind_mip, cmap='hot', aspect='equal')
pr_axes[1].set_title("injection density MaxIP")

pr_axes[2].imshow(inf_mip, cmap='hot', aspect='equal')
pr_axes[2].set_title("injection fraction MaxIP")

plt.show()

In [ ]:
# compute the maximum intensity projection (along the anterior-posterior axis) of the projection data
pd_mip = pd.max(axis=0)
ind_mip = ind.max(axis=0)
inf_mip = inf.max(axis=0)

# show that slice of all volumes side-by-side
f, pr_axes = plt.subplots(1, 3, figsize=(15, 6))

pr_axes[0].imshow(pd_mip, cmap='hot', aspect='equal')
pr_axes[0].set_title("projection density MaxIP")

pr_axes[1].imshow(ind_mip, cmap='hot', aspect='equal')
pr_axes[1].set_title("injection density MaxIP")

pr_axes[2].imshow(inf_mip, cmap='hot', aspect='equal')
pr_axes[2].set_title("injection fraction MaxIP")

plt.show()

In [ ]:
#get pixel coordinates of each ROI, then apply as mask take projection density 
np.unique(df[300])

In [ ]:
np.unique(annot[300])

In [ ]:
df.shape